In [24]:
from pyspark.sql.functions import when, col, udf, split, trim
from pyspark.sql.types import StringType

# Ensure the below library is installed on the fabric environment
import reverse_geocoder as rg

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 28, Finished, Available, Finished)

In [2]:
df = spark.read.table("earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 6, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e3bfbec4-a51c-4940-8d0b-c4d533f06044)

In [9]:
def get_country_code(lat,lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """

    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 13, Finished, Available, Finished)

In [11]:
# Registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 15, Finished, Available, Finished)

In [12]:
# adding country_code
df_with_location = df.withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))
display(df_with_location)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e58f98ac-6e21-404f-b2df-f9f8a150c258)

In [13]:
# adding significance classification
df_with_location_sig_class = df_with_location.withColumn(
    'sig_class',
    when(col("sig") < 100, "Low")\
    .when((col("sig") >= 100) & (col("sig") < 500), "Moderate")\
    .otherwise("High")
)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 17, Finished, Available, Finished)

In [14]:
display(df_with_location_sig_class)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, be5a473e-6511-4d2a-8fb6-cefec00b0a6f)

In [25]:
# splitting the place_description column to place and country
df_with_location_sig_class = df_with_location_sig_class.withColumn("country", split(col("place_description"), ',')[1])\
                                .withColumn("place_description", split(col("place_description"), ",")[0])\
                                .withColumn("title", trim(split(col("title"), "-")[0]))
display(df_with_location_sig_class)

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8436c5f4-8b2e-406d-bf85-6274fe699679)

In [26]:
# appending the data to the gold table

df_with_location_sig_class.write.mode("append").saveAsTable('earthquake_events_gold')

StatementMeta(, ac7a86db-9db5-480e-9c22-42503d75dd9f, 30, Submitted, Running, Running)